# Neural networks

## Miguel Ángel Canela, IESE Business School

******

###  What is a neural network?

An (artificial) **neural network** is an interconnected set of computational elements, called **nodes** or neurons, organized in **layers**. A connection of a node to another node has a **weight**. In machine learning, these weights are learned from training data. 

There many types of neural networks, but we are interested here in the **multilayer perceptron** (MLP). MLP models can be used both in regression and in classification. In scikit-learn, they are implemented in the classes `MLPRegressor` and `MLPClassifier` of the module `neural_networks`. Although the field of neural networks is very rich and diverse (see Haykin, 1999, for a general presentation), practitioners usually refer to the multilayer perceptron when using the expressions neural network or artificial neural network (ANN). An MLP model can be seen as a set of equations, as we see in the next section.

The advantage of neural networks is the robustness to noisy data. The network can learn to work around the uninformative or erroneous examples in the data set. The main drawbacks are that they are opaque to human intuition, take longer training time than other methods, and are prone to overfitting.

### The multilayer perceptron

A multilayer perceptron is formed by:

* The **input layer**, whose nodes are the features used for prediction.

* The **output layer**, which in regression has a unique node, which is the target variable, and in classification has one node for every class.

* Some **hidden layers**. If the network is **fully-conected**, that is, if every node of a layer is connected to all nodes of the following layer, the model is completely specified by the number of hidden layers and the number of nodes in each hidden layer. We only consider fully-connected networks in this note.

How do we use these networks as predictive models? Suppose that $Z$ is a node of a hidden layer and $U_1, \dots,U_k$ are the nodes of the preceding layer. Then, the values of $Z$ are calculated as

$$Z = F\big(w_0 + w_1 U_1 + w_2 U_2 + \cdots + w_k U_k\big).$$

The slope coefficients $w_1$, $w_2$, $\dots$, $w_k$ are called **weights**, and the intercept $w_0$ is called **bias**. $F$ is the **activation function**. The activation introduces nonlinearity in the model. Just a few years ago, the **logistic function**,

$$F(x) = {1\over 1 + \exp(–x)},$$

was the recommended activation function in most references. Some prefer the **hyperbolic tangent function**, 

$$\tanh(x) = {\exp(x) - \exp(–x)\over \exp(x) + \exp(–x)},$$

but the current trend favors the **rectified linear unit function** (ReLU), which is the default in scikit-learn. ${\rm ReLU}(x)$ is equal to $x$ when $x\ge 0$ and equal to 0 otherwise.

The model of Figure 1 is a set of three equations. The two equations that allow us to go from the input layer to the hidden layer combine the features with weights $w_{1A}$, $w_{2A}$ and $w_{3A}$ and $w_{1B}$, $w_{2B}$ and $w_{3B}$, respectively. The biases are $w_{0A}$, and $w_{0B}$, respectively. This model has a total of 11 parameters.

In A and B, activation  is applied to the values given by these equations. Once the activation has been applied, A and B are combined in the third equation with weights $w_{AY}$ and $w_{BY}$ and bias $w_{0Y}$, to obtain the predicted value for $Y$.

******

### Figure 1

![](fig.jpg)

******

In an MLP regressor, there is no activation at the output node, so the equation predicting the values at this node is linear. In an MLP classifier, the final output is a vector containing the **class probabilities**, as returned by an activation function called the **softmax function**, which works as follows. 

Suppose that there are three classes and that the output sent by the hidden layer is the vector $[z_1, z_2, z_3]$. Then the softmax function will return the three following probabilities, one for every for class:
$$p_1 = {e^{z_1}\over e^{z_1}+e^{z_2}+e^{z_3}}\,,\qquad
p_2 = {e^{z_2}\over e^{z_1}+e^{z_2}+e^{z_3}}\,,\qquad
p_3 = {e^{z_3}\over e^{z_1}+e^{z_2}+e^{z_3}}\,.$$

Note that the sum of the probabilities is 1. The predicted class is the class with the highest probability. In many places, the graphical representation of an MLP classifier puts the classes as the nodes of the output layer, instead of the target variable as the unique output node.

The multilayer perceptron could be seen as if the instances were circulating through the network one-by-one. The feature values are entered in the input nodes, which send them to the two nodes of the first hidden layer. At each hidden node, they are combined using the corresponding weights, and the result is transformed with the activation function. The hidden nodes send the resulting values to the nodes of the next layer, where they are combined. According to the legend, this simulates how animal neurons learn. 

### Technical details

Once the general description of how the regression/classification is performed with a perceptron has been presented, let me briefly comment some technical issues:

* *The number of nodes of the hidden layers*. Most computer implementations have a default rule to set this number. The ruler takes into account the number of inputs. In most cases there is not much benefit in changing this. Mind that the more nodes, the more parameters has the model, which may lead to overfitting. 

* *How to find the optimal weights*. This typically follows a method called **backpropagation**. Initially, the weights are randomly assigned. Then, an iterative process starts. At each step, the prediction is performed with the actual weights, the value of a **loss function** is calculated and the weights are adjusted in order to reduce the loss. The process should converge to an optimal solution, but, in practice, the maximum number of passes of the data set is prespecified. In some places (e.g. in TensorFlow) the passes are called **epochs**. In scikit-learn, they are just called iterations. In regression, the loss is the sum of the squared errors, while in classification is the **cross-entropy**. 

* *The optimization method*. The learning process uses a specific optimization method, called `solver` in scikit-learn (`optimizer` in TensorFlow), to minimize the loss function. LBFGS has been the choice for many years, so you can still find its traces in this context. The current default of scikit-learn is `solver=’adam’`, which uses a variant of a well known method, the **stochastic gradient descent**. For small datasets, however, LFBGS can converge faster and perform better.

* *The batch size*. In the stochastic gradient descent method, we modify the parameters little by little based on the current loss value on a random **batch** of data. In every epoch, the training data are randomly partitioned in batches and all the batches are tried. The default batch size in scikit-learn is 200.

* *Normalization*. The multilayer perceptron is sensitive to feature scaling, so it is highly recommended to scale your data. In the old data mining suites, normalization was applied as a part of the algorithm, and the output was scaled back to the original range. It is not so in the modern languages like R and Python. In scikit-learn, normalization is left to the user. **Min-max normalization** is typical also in this context. 

* *How to specify the parameters that control the learning process*. The **learning rate** is a parameter that controls how fast the adjustment of the weights is done. If it is too low, there is no convergence to the optimal solution. If it is too high, we can overshoot the optimal solution. The **momentum** controls the inertia: high momentum values influence the adjustment to the current weight to have the same direction as the previous adjustment. In scikit-learn, momentum is only used with the stochastic gradient descent solvers.

### References

1. S Haykin (1999), *Neural Networks*, Prentice Hall.

2. F Chollet (2017), *Deep Learning with Python*, Manning.

3. A Géron (2017), *Hands-On Machine Learning with Scikit-Learn & TensorFlow*, O'Reilly.

4. scikit-learn user guide (2015).